## Extending ASDF
Many simple data stuctures can be stored using the core objects supported by all ASDF implementations. For domain-specific and other custom structures asdf supports an extension system that allows transparent saving and loading of custom objects.

In [ ]:
import os
import numpy as np
import asdf

np.random.seed(42)

def print_file(fn):
    """
    A helper function to print out an ASDF file
    """
    with open(fn, "r", encoding="unicode_escape") as f:
        print(f.read())

## Extension overview

For most users installing an extension is as easy as `pip install my_asdf_extension`.

For developers please see the [documentation](https://asdf.readthedocs.io/en/latest/asdf/extending/extensions.html) to get started developing an extension.

Extensions include:
- code that maps custom objects to other supported structures
- tags that annotate these structures
- schemas that constrain these structures

# ⭐ The asdf-astropy extension

As mentioned above, asdf provides an extension API that can be used to save custom objects to ASDF files. [asdf-astropy](https://pypi.org/project/asdf-astropy/) is an ASDF extension that supports many [Astropy](https://www.astropy.org/) objects. To set up the extension all you need to do is `pip install asdf-astropy`.

Once the extension is installed many [Astropy](https://www.astropy.org/) objects will be supported including:

- [unit](https://docs.astropy.org/en/stable/units/ref_api.html#module-astropy.units) and [quantity](https://docs.astropy.org/en/stable/units/quantity.html) objects
- (most) [modeling](https://docs.astropy.org/en/stable/modeling/index.html) objects
- [time](https://docs.astropy.org/en/stable/time/index.html) objects
- [coordinate](https://docs.astropy.org/en/stable/coordinates/index.html) objects
- [tables](https://docs.astropy.org/en/stable/table/index.html)

For example, to save an astropy [Table](https://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) simply add it to the ASDF tree:

In [ ]:
import asdf
from astropy.table import Table

af = asdf.AsdfFile()
af["table"] = Table(dtype=[("a", "f4"), ("b", "i4"), ("c", "S2")])
af.write_to("table.asdf")
print_file("table.asdf")

Notice above that the Table is broken down into several nested "tagged" mappings. When loaded asdf will reconstruct the Table without the user needing to be aware of any of this serialization and deserialization

In [ ]:
af = asdf.open("table.asdf")
print(type(af["table"]))
af["table"]

# Exercise: Saving Astropy objects

Write an ASDF file containing the following `astropy` objects:
1. [Quantity](https://docs.astropy.org/en/stable/units/quantity.html)
2. A [model](https://docs.astropy.org/en/stable/api/astropy.modeling.Model.html#astropy.modeling.Model)

   Hint: The [astropy.modeling](https://docs.astropy.org/en/stable/modeling/index.html) package provides a framework for representing models and performing model evaluation and fitting. Models are initialized using their parameters like in the following example for [Gaussian1D](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D):
   ```
   from astropy.modeling import models
   gauss = models.Gaussian1D(amplitude=10, mean=3, stddev=1.2)
   ```
3. A [Time](https://docs.astropy.org/en/stable/time/index.html) object

    Hint: The [astropy.time](https://docs.astropy.org/en/stable/time/ref_api.html#module-astropy.time) package provides functionality for manipulating times and dates. To initialize it supply a string and a [format](https://docs.astropy.org/en/stable/time/index.html#id3), or supply a datetime object.
    
4. A [ICRS](https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html) coordinate object.

## The roman_datamodels extension

Roman files make use of an asdf extension provided by roman_datamodels to allow reading and writing roman Data Models. This will largely be transparent to the user (especially if they use the data model API provided by roman_datamodels) but can be seen by inspecting types in the example file.

In [ ]:
af = asdf.open("../data/roman.asdf")
type(af["roman"])

Although the value stored under the "roman" key behaves like a dictionary it is an instance of a special "node" class. The same is true for some of the nested structures.

In [ ]:
type(af["roman"]["meta"]["exposure"])

These special nodes can be useful for inspecting associated schemas for determining allowed values.

In [ ]:
exposure_schema = af["roman"]["meta"]["exposure"].get_schema()
exposure_schema["properties"]["type"]

## Other extensions
For more examples see the following packages which provide asdf extensions supporting other custom types:
- gwcs
- dkist
- sunpy
- asdf-zarr